# Artificial Intelligence
## L.EIC – 3rd Year/2nd Semester
### Exercise Sheet 1
# Solving Problems by Searching

## The Two Buckets Problem

<img src="https://qph.cf2.quoracdn.net/main-qimg-45726b16b460cae0147ae8ca245a8fb0-pjlq" width="250px" height="250px" align="right">

Two buckets of capacities **c1** (e.g. 4 liters) and **c2** (e.g. 3 liters), respectively, are initially empty. The buckets do not have any intermediate markings. The only operations you can perform are:

- Fill (completely) a bucket 
- Empty a bucket.
- Pour one bucket into the other (until the target one is full or the source one is empty).

The aim is to determine which operations to carry out so that the first bucket contains exactly **n** liters (e.g. 2 litres).

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.

What is the size of the state space for this problem? Represent the state space by drawing the possible objective states and displaying some of the possible transitions from the initial state.

Solve the problem by hand, using tree search. What solutions have you found?

### Buildind a computational approach to handle the problem

To build a program to solve the buckets problem, we will implement a solution that separates the problem definition from the algorithms used to traverse the state space. This way, we can reuse our implementations of the search strategies in other problems.

#### Representing the two buckets problem as a search problem

Let's start by defining a state for the buckets problem. For that, it'll suffice to aggregate two quantities, each representing the amount of water in one of the buckets. We also define a way of printing the state.

In [1]:
class BucketState:
    c1 = 4   # capacity for bucket 1
    c2 = 3   # capacity for bucket 2
    
    def __init__(self, b1, b2):
        self.b1 = b1
        self.b2 = b2

    '''needed for the visited list'''
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False

    def __ne__(self, other):
        """Overrides the default implementation (unnecessary in Python 3)"""
        return not self.__eq__(other)
    
    def __hash__(self):
        return hash((self.b1, self.b2)) 
    ''' - '''

    def __str__(self):
        return "(" + str(self.b1) + ", " + str(self.b2) + ")"

Now we define each of the operators on states:

In [2]:
# emptying the first bucket
def empty1(state):
    if state.b1 > 0:
        return BucketState(0, state.b2)
    return None

# emptying the second bucket
def empty2(state):
    # your code here
    if state.b2 > 0:
        return BucketState(state.b1, 0)
    return None
    

# your code here
def fill1(state):
    if state.b1 < state.c1:
        return BucketState(state.c1, state.b2)
    return None

def fill2(state):
    if state.b2 < state.c2:
        return BucketState(state.b1, state.c2)
    return None

def pour12_empty1(state):
    if (state.b1 > 0) and (state.b2 < state.c2) and (state.b1 + state.b2 <= state.c2):
        return BucketState(0, state.b1 + state.b2)
    return None

def pour21_empty2(state):
    if (state.b2 > 0) and (state.b1 < state.c1) and (state.b1 + state.b2 <= state.c1):
        return BucketState(state.b1 + state.b2, 0)
    return None

def pour12_fill2(state):
    if (state.b1 > 0) and (state.b2 < state.c2):
        if (state.b1 + state.b2 > state.c2):
            return BucketState( state.b1 + state.b2 - state.c2, state.c2)
        return BucketState(0, state.b1 + state.b2)

def pour21_fill1(state):
    if (state.b2 > 0) and (state.b1 < state.c1):
        if (state.b1 + state.b2 > state.c1):
            return BucketState(state.c1,  state.b1 + state.b2 - state.c1)
        return BucketState(state.b1 + state.b2, 0)



The following function will aggregate all states that can be generated from a given one:

In [3]:
def child_bucket_states(state):
    new_states = []
    if(empty1(state)):
        new_states.append(empty1(state))
    if(empty2(state)):
        new_states.append(empty2(state))
    if(fill1(state)):
        new_states.append(fill1(state))
    if(fill2(state)):
        new_states.append(fill2(state))
    if(pour12_fill2(state)):
        new_states.append(pour12_fill2(state))
    if(pour12_empty1(state)):
        new_states.append(pour12_empty1(state))
    if(pour21_fill1(state)):
        new_states.append(pour21_fill1(state))
    if(pour21_empty2(state)):
        new_states.append(pour21_empty2(state))
    return new_states

Play around with the state transition operators and check if they are working properly:

In [4]:
s = BucketState(0, 0)
s = fill1(s)
s = pour12_fill2(s)
print(s)

states = child_bucket_states(BucketState(0, 0))
for i in range(len(states)):
    print(states[i])

# your code here


(1, 3)
(4, 0)
(0, 3)


Finally, we need to define the goal condition:

In [5]:
def goal_bucket_state(state):
    # your code here
    if (state.b1 == 2):
        return True
    return False

Test your goal condition:

In [6]:
# your code here
state = BucketState(2, 1)
print( goal_bucket_state(state) )

state = BucketState(1, 3)
print( goal_bucket_state(state) )


True
False


#### Implementing search algorithms

Let us start by defining an appropriate structure to represent a node in a search tree. Each tree node will include:
- a state of the problem
- a link to its parent (to allow traveling from a leaf node towards the root of the tree)
- a list of child nodes

In [7]:
# A generic definition of a tree node holding a state of the problem
class TreeNode:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        if self.parent is None:
            self.depth = 0
            self.cost = 0
        else:
            self.depth = self.parent.depth + 1
            

    def add_child(self, child_node):
        self.children.append(child_node)
        child_node.parent = self
    
    

##### Breadth-first search

Based on this structure, we can now implement breadth-first search. Note that we want the implementation to be independent of the problem at hand (in this case, the two buckets problem).

In [8]:
from collections import deque

def breadth_first_search(initial_state, goal_state_func, operators_func):
    root = TreeNode(initial_state)   # create the root node in the search tree
    queue = deque([root])   # initialize the queue to store the nodes
    
    while queue:
        node = queue.popleft()   # get first element in the queue
        if goal_state_func(node.state):   # check goal state
            return node
        
        for state in operators_func(node.state):   # go through next states
            # create tree node with the new state
            # your code here
            child = TreeNode(state, node)
            
            # link child node to its parent in the tree
            # your code here
            # child.parent = node  this is done in the constructor so it's not needed here
            
            # enqueue the child node
            # your code here
            queue.append(child)
            

    return None

We can now use this function to actually perform a breadth-first search on the buckets problem: we pass it the initial state, our goal condition function, and the function for obtaining child states.

In [9]:
goal = breadth_first_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states)
print(goal.state)

(2, 3)


In order to print the actual steps from the initial state to the last, we can take advantage of each node's link to its parent.

In [10]:
def print_solution(node):
    # your code here
    print(node.state)

    if (node.parent != None):
        return print_solution(node.parent)

Now we can print the solution:

In [11]:
print_solution(goal)

(2, 3)
(4, 1)
(0, 1)
(1, 0)
(1, 3)
(4, 0)
(0, 0)


If we need a description for each of the employed operators, we could have each operation function return also such a description, and modify the TreeNode class so that each node also includes a description of the edge to get there. We leave that as an exercise after class.

##### Depth-first search

Implement depth-first search (again, in a manner that is independent of the problem at hand). You can start from your breadth-first search implementation and with minor changes get an implementation for depth-first search.

In [12]:
def depth_first_search(initial_state, goal_state_func, operators_func):
    # your code here
    root = TreeNode(initial_state)
    stack = deque([root])
    states_visited = set()  

    while stack:
        node = stack.pop()
        if goal_state_func(node.state):
            return node

        for state in operators_func(node.state):
            if state in states_visited:
                continue
            child = TreeNode(state, node)
            stack.append(child)
            states_visited.add(state)

# I used a set of states visited to avoid infinite loops, but the most efficient way would probably be to 
# check if a certain state had a parent that was equal to itself, which would mean that it was a loop
# If it was done that way, the set of states visited would not be needed, hence it would be more efficient in terms of memory usage

Test it on the two buckets problem.

In [13]:
# your code here

goal = depth_first_search(BucketState(0,0), goal_bucket_state, child_bucket_states)

print_solution(goal)


(2, 0)
(0, 2)
(4, 2)
(3, 3)
(3, 0)
(0, 3)
(0, 0)


If you are unable to get a solution, think about it: depth-first search is not a complete search method, and one of the reasons for that is if the state space contains cycles. As such, you need to make sure you avoid entering into a cycle by keeping a visited nodes list or set and checking that list whenever you generate a new state.

##### Depth-limited search

Another way to make it work is to impose a depth limit to the problem. Implement depth-limited search.

In [14]:
def depth_limited_search(initial_state, goal_state_func, operators_func, depth_limit):
    # your code here
    
    root = TreeNode(initial_state)
    stack = deque([root])

    while stack:
        node = stack.pop()
        if goal_state_func(node.state):
            return node

        if node.depth < depth_limit:
            for state in operators_func(node.state):
                child = TreeNode(state, node)
                stack.append(child)

Test it on the two buckets problem.

In [15]:
# your code here
goal = depth_limited_search(BucketState(0,0), goal_bucket_state, child_bucket_states, 6)

print_solution(goal)


(2, 0)
(0, 2)
(4, 2)
(3, 3)
(3, 0)
(0, 3)
(0, 0)


##### Iterative deepening search

Based on depth-limited, you can easily implement iterative-deepening search.

In [16]:
def iterative_deepening_search(initial_state, goal_state_func, operators_func, depth_limit):
    # your code here
    
    for i in range(depth_limit):
        goal = depth_limited_search(initial_state, goal_state_func, operators_func, i)
        if goal != None:
            return goal
        
    

Again, test it on the two buckets problem.

In [17]:
# your code here
goal = iterative_deepening_search(BucketState(0,0), goal_bucket_state, child_bucket_states, 10)

print_solution(goal)

(2, 0)
(0, 2)
(4, 2)
(3, 3)
(3, 0)
(0, 3)
(0, 0)


##### Greedy search

Start by defining an heuristic function and implement the Greedy Search algorithm (independent of the problem at hand)

In [18]:
def heuristic_bucket(node):
    # heuristic function for the bucket filling problem
    
    # your code here
    
    return abs(node.state.b1 - 2)  # + abs(node.state.b2 - 0)

In [19]:
def greedy_search(initial_state, goal_state_func, operators_func, heuristic_func):
    # your code here
    
    root = TreeNode(initial_state)
    queue = deque([root])
    states_visited = set()

    while queue:
        node = queue.popleft()
        if goal_state_func(node.state):
            return node

        for state in operators_func(node.state):
            if state in states_visited:
                continue
            child = TreeNode(state, node)
            child.heuristic = heuristic_func(child)
            states_visited.add(state)
            queue.append(child)
            queue = deque(sorted(queue, key=lambda x: x.heuristic))


    


Test it on the two buckets problem.

In [20]:
# your code here
goal = greedy_search(BucketState(0,0), goal_bucket_state, child_bucket_states, heuristic_bucket)

print_solution(goal)

(2, 3)
(4, 1)
(0, 1)
(1, 0)
(1, 3)
(4, 0)
(0, 0)


##### A* Algorithm

Reuse the heuristic function defined before. This is very similar to greedy search, the difference is that it takes into account the cost of the path so far

In [21]:
def a_star_search(initial_state, goal_state_func, operators_func, heuristic):
    # your code here
    
    root = TreeNode(initial_state)
    queue = deque([root])
    states_visited = set()

    while queue:
        node = queue.popleft()
        if goal_state_func(node.state):
            return node

        for state in operators_func(node.state):
            if state in states_visited:
                continue
            child = TreeNode(state, node)
            child.heuristic = heuristic(child)
            child.cost = node.cost + 1
            states_visited.add(state)
            queue.append(child)
            queue = deque(sorted(queue, key=lambda x: x.heuristic + x.cost)) 

Again, test it on the two buckets problem.

In [22]:
# your code here

goal = a_star_search(BucketState(0,0), goal_bucket_state, child_bucket_states, heuristic_bucket)

print_solution(goal)

(2, 3)
(4, 1)
(0, 1)
(1, 0)
(1, 3)
(4, 0)
(0, 0)


## The Missionaries and Cannibals Problem 

<img src="https://www.gamezkingdom.com/content/images/thumbs/0002926_missionaries-and-cannibals.jpeg" width="250px" height="250px" align="right">

Three missionaries and three cannibals are on one of the banks of the river with a boat that only takes one or two people. The boat cannot travel the river alone.

The goal is to find a way to get the six to the other bank of the river without ever leaving more cannibals than missionaries on one of the banks (even at the instant they leave/join the boat) during the process.

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.

Solve the problem by hand, using tree search. What solutions have you found?

Represent the problem as a search problem and take advantage of the implemented search algorithms to find solutions!

In [23]:
class MissionariesState:
    
    def __init__(self, m1, c1, m2, c2, boat):
        self.m1 = m1
        self.c1 = c1
        self.m2 = m2
        self.c2 = c2
        self.boat = boat

    '''needed for the visited list'''
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False

    def __ne__(self, other):
        """Overrides the default implementation (unnecessary in Python 3)"""
        return not self.__eq__(other)
    
    def __hash__(self):
        return hash((self.m1, self.c1, self.m2, self.c2, self.boat)) 
    ''' - '''

    def __str__(self):
        return "(" + str(self.m1) + ", " + str(self.c1) + ", " + str(self.m2) + ", " + str(self.c2) + ", " + str(self.boat) + ")"


def move_missionary_1_2(state):
    if state.m1 > 0 and (state.m1-1 >= state.c1 or state.m1-1 == 0) and (state.m2+1 >= state.c2) and state.boat == 1:
        return MissionariesState(state.m1-1, state.c1, state.m2+1, state.c2, 0)
    return None

def move_missionary_2_1(state):
    if state.m2 > 0 and (state.m2-1 >= state.c2 or state.m2-1 == 0) and (state.m1+1 >= state.c1) and state.boat == 0:
        return MissionariesState(state.m1+1, state.c1, state.m2-1, state.c2, 1)
    return None

def move_cannibal_1_2(state):
    if state.c1 > 0 and (state.m2 >= state.c2 + 1 or state.m2 == 0) and state.boat == 1:
        return MissionariesState(state.m1, state.c1-1, state.m2, state.c2+1, 0)
    return None

def move_cannibal_2_1(state):
    if state.c2 > 0 and (state.m1 >= state.c1 + 1 or state.m1 == 0) and state.boat == 0:
        return MissionariesState(state.m1, state.c1+1, state.m2, state.c2-1, 1)
    return None

def move_2_missionaries_1_2(state):
    if state.m1 > 1 and (state.m1-2 >= state.c1 or state.m1-2 == 0) and state.boat == 1:
        return MissionariesState(state.m1-2, state.c1, state.m2+2, state.c2, 0)
    return None

def move_2_missionaries_2_1(state):
    if state.m2 > 1 and (state.m2-2 >= state.c2 or state.m2-2 == 0) and state.boat == 0:
        return MissionariesState(state.m1+2, state.c1, state.m2-2, state.c2, 1)
    return None

def move_2_cannibals_1_2(state):
    if state.c1 > 1 and (state.m2 >= state.c2 + 2 or state.m2 == 0) and state.boat == 1:
        return MissionariesState(state.m1, state.c1-2, state.m2, state.c2+2, 0)
    return None

def move_2_cannibals_2_1(state):
    if state.c2 > 1 and (state.m1 >= state.c1 + 2 or state.m1 == 0) and state.boat == 0:
        return MissionariesState(state.m1, state.c1+2, state.m2, state.c2-2, 1)
    return None

def move_both_1_2(state):
    if state.m1 > 0 and state.c1 > 0 and state.m2 >= state.c2 and state.boat == 1:
        return MissionariesState(state.m1-1, state.c1-1, state.m2+1, state.c2+1, 0)
    return None

def move_both_2_1(state):
    if state.m2 > 0 and state.c2 > 0 and state.m1 >= state.c1 and state.boat == 0:
        return MissionariesState(state.m1+1, state.c1+1, state.m2-1, state.c2-1, 1)
    return None

def child_missionaries_states(state):
    new_states = []
    if (move_missionary_1_2(state)):
        new_states.append(move_missionary_1_2(state))
    
    if (move_missionary_2_1(state)):
        new_states.append(move_missionary_2_1(state))
    
    if (move_cannibal_1_2(state)):
        new_states.append(move_cannibal_1_2(state))

    if (move_cannibal_2_1(state)):
        new_states.append(move_cannibal_2_1(state))
    
    if (move_2_missionaries_1_2(state)):
        new_states.append(move_2_missionaries_1_2(state))

    if (move_2_missionaries_2_1(state)):
        new_states.append(move_2_missionaries_2_1(state))
    
    if (move_2_cannibals_1_2(state)):
        new_states.append(move_2_cannibals_1_2(state))
    
    if (move_2_cannibals_2_1(state)):
        new_states.append(move_2_cannibals_2_1(state))

    if (move_both_1_2(state)):
        new_states.append(move_both_1_2(state))
    
    if (move_both_2_1(state)):
        new_states.append(move_both_2_1(state))

    return new_states


def goal_missionaries_state(state):
    return state.m2 == 3 and state.c2 == 3 and state.m1 == 0 and state.c1 == 0 and state.boat == 0

def print_solution_missionaries(node):
    if (node.parent != None):
        print_solution_missionaries(node.parent)
        print(node.state)
    else:
        print(node.state)

def heuristic_missionaries(node):
    return node.state.m1 + node.state.c1


# goal = breadth_first_search(MissionariesState(3, 3, 0, 0, 1), goal_missionaries_state, child_missionaries_states)

# goal = depth_first_search(MissionariesState(3, 3, 0, 0, 1), goal_missionaries_state, child_missionaries_states)

# goal = iterative_deepening_search(MissionariesState(3, 3, 0, 0, 1), goal_missionaries_state, child_missionaries_states, 10)

# goal = greedy_search(MissionariesState(3, 3, 0, 0, 1), goal_missionaries_state, child_missionaries_states, heuristic_missionaries)

goal = a_star_search(MissionariesState(3, 3, 0, 0, 1), goal_missionaries_state, child_missionaries_states, heuristic_missionaries)

print_solution_missionaries(goal)


(3, 3, 0, 0, 1)
(3, 1, 0, 2, 0)
(3, 2, 0, 1, 1)
(3, 0, 0, 3, 0)
(3, 1, 0, 2, 1)
(1, 1, 2, 2, 0)
(2, 2, 1, 1, 1)
(0, 2, 3, 1, 0)
(0, 3, 3, 0, 1)
(0, 1, 3, 2, 0)
(1, 1, 2, 2, 1)
(0, 0, 3, 3, 0)


## N-Puzzle Problem

The objective of this exercise is the application of search methods, with emphasis on informed
search methods and the A\* algorithm, to solve the well-known N-Puzzle problem. The desired
objective self for the puzzle is as follows (0 represents the empty space):

<table>
<tr><th>9Puzzle</th><th>16Puzzle</th></tr>
<tr>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 2   | 3   |
| 4   | 5   | 6   |
| 7   | 8   | 0   |


</td>
<td>

|     |     |     |     |
| --- | --- | --- | --- |
| 1   | 2   | 3   | 4   |
| 5   | 6   | 7   | 8   |
| 9   | 10  | 11  | 12  |
| 13  | 14  | 15  | 0   |

</td>
</tr>
</table>

Starting from a given initial state, the goal is to determine which operations to perform to
solve the puzzle, reaching the desired objective self.

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.

Represent the problem as a search problem and take advantage of the implemented search algorithms to find solutions!

For the Greedy Search and the A* Algorithm suppose the following heuristics for these methods:
- H1 - Number of incorrect placed pieces;
- H2 - Sum of manhattan distances from incorrect placed pieces to their correct places. 

Finally Compare the results obtained concerning execution time and memory space occupied in solving the following problems using the previous methods

<table>
<tr><th>Prob. 1</th><th>Prob. 2</th><th>Prob. 3</th><th>Prob. 4</th></tr>
<tr>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 2   | 3   |
| 5   | 0   | 6   |
| 4   | 7   | 8   | 

</td>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 3   | 6   |
| 5   | 2   | 0   |
| 4   | 7   | 8   | 

</td>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 6   | 2   |
| 5   | 7   | 3   |
| 0   | 4   | 8   | 

</td>
<td>

|     |     |     |     |
| --- | --- | --- | --- |
| 5   | 1   | 3   | 4   |
| 2   | 0   | 7   | 8   |
| 10  | 6   | 11  | 12  |
| 9   | 13  | 14  | 15  |

</td>
</tr>
</table>

In [38]:
# the following arrays represent the problem for testing
import copy

# 5x5 board is too long to solve
initial_states = [
    [[1, 2, 3], [5, 0, 6], [4, 7, 8]],
    [[1, 3, 6], [5, 2, 0], [4, 7, 8]],
    [[1, 6, 2], [5, 7, 3], [0, 4, 8]],
    [[5, 1, 3, 4], [2, 0, 7, 8], [10, 6, 11, 12], [9, 13, 14, 15]],
    [[3, 2, 1, 6, 5], [4, 14, 8, 7, 10], [9, 18, 0, 12, 15], [13, 22, 19, 23, 16], [17, 21, 24, 20, 11]],
    [[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 13, 14, 15], [16, 17, 0, 18, 19], [21, 22, 23, 24, 20]]
    
]

class NPuzzleState:
    
    def __init__(self, board):
        self.board = board
        

    '''needed for the visited list'''
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False

    def __ne__(self, other):
        """Overrides the default implementation (unnecessary in Python 3)"""
        return not self.__eq__(other)
    
    def __hash__(self):
        return hash(str(self.board)) 
    ''' - '''

    def __str__(self):
        board_string = ""
        for row in self.board:
            board_string += str(row) + "\n"
        return board_string


def move_zero_up(state):
    for i in range(len(state.board)):
        for j in range(len(state.board[i])):
            if (state.board[i][j] == 0) and (i > 0):
                new_board = copy.deepcopy(state.board)
                new_board[i][j] = new_board[i-1][j]
                new_board[i-1][j] = 0
                return NPuzzleState(new_board)
    return None

def move_zero_down(state):
    for i in range(len(state.board)):
        for j in range(len(state.board[i])):
            if (state.board[i][j] == 0) and (i < (len(state.board)-1)):
                new_board = copy.deepcopy(state.board)
                new_board[i][j] = new_board[i+1][j]
                new_board[i+1][j] = 0
                return NPuzzleState(new_board)
    return None

def move_zero_left(state):
    for i in range(len(state.board)):
        for j in range(len(state.board[i])):
            if (state.board[i][j] == 0) and (j > 0):
                new_board = copy.deepcopy(state.board)
                new_board[i][j] = new_board[i][j-1]
                new_board[i][j-1] = 0
                return NPuzzleState(new_board)
    return None

def move_zero_right(state):
    for i in range(len(state.board)):
        for j in range(len(state.board[i])):
            if (state.board[i][j] == 0) and (j < (len(state.board[i])-1)):
                new_board = copy.deepcopy(state.board)
                new_board[i][j] = new_board[i][j+1]
                new_board[i][j+1] = 0
                return NPuzzleState(new_board)
    return None

def child_npuzzle_states(state):
    new_states = []
    if (move_zero_up(state)):
        new_states.append(move_zero_up(state))
    if (move_zero_down(state)):
        new_states.append(move_zero_down(state))
    if (move_zero_left(state)):
        new_states.append(move_zero_left(state))
    if (move_zero_right(state)):
        new_states.append(move_zero_right(state))
    return new_states

def goal_npuzzle_state(state):
    for i in range(len(state.board)):
        for j in range(len(state.board[i])):
            if (state.board[i][j] == 0):
                continue
            if (state.board[i][j] != (i*len(state.board[i])+j+1)):
                return False
    return True        
    
    
def print_solution_npuzzle(node):
    if (node.parent != None):
        print_solution_npuzzle(node.parent)
        print(node.state)
    else:
        print(node.state)

def heuristic_npuzzle1(node):
    state = node.state
    counter = 0
    for i in range(len(state.board)):
        for j in range(len(state.board[i])):
            if (state.board[i][j] == 0):
                continue
            if (state.board[i][j] != (i*len(state.board[i])+j+1)):
                counter += 1
    return counter

def heuristic_npuzzle2(node):
    state = node.state
    counter = 0
    for i in range(len(state.board)):
        for j in range(len(state.board[i])):
            x = state.board[i][j] 
            if (x == 0):
                continue
            if (x != (i*len(state.board[i])+j+1)):
                correct_pos_i = x % len(state.board[i])
                correct_pos_j = x // len(state.board[i])
                counter += abs(correct_pos_i - i) + abs(correct_pos_j - j)
    return counter
    
    
# Breath First Search -> initial_states[2] 2.4s  and initial_states[3] 28.7s
# goal = breadth_first_search(NPuzzleState(initial_states[3]), goal_npuzzle_state, child_npuzzle_states)

# goal = depth_first_search(NPuzzleState(initial_states[0]), goal_npuzzle_state, child_npuzzle_states)

# goal = iterative_deepening_search(NPuzzleState(initial_states[0]), goal_npuzzle_state, child_npuzzle_states, 11)

# goal = greedy_search(NPuzzleState(initial_states[0]), goal_npuzzle_state, child_npuzzle_states, heuristic_npuzzle1)

# goal = greedy_search(NPuzzleState(initial_states[0]), goal_npuzzle_state, child_npuzzle_states, heuristic_npuzzle2)

# A Star Search 1st heuristic: initial_states[2] 0.1s  and initial_states[3] 0.1s
# goal = a_star_search(NPuzzleState(initial_states[4]), goal_npuzzle_state, child_npuzzle_states, heuristic_npuzzle1)

# A Star Search 2nd heuristic: initial_states[2] and initial_states[3] just as fast as the first heuristic
goal = a_star_search(NPuzzleState(initial_states[3]), goal_npuzzle_state, child_npuzzle_states, heuristic_npuzzle2)

print_solution_npuzzle(goal)


[5, 1, 3, 4]
[2, 0, 7, 8]
[10, 6, 11, 12]
[9, 13, 14, 15]

[5, 1, 3, 4]
[2, 6, 7, 8]
[10, 0, 11, 12]
[9, 13, 14, 15]

[5, 1, 3, 4]
[2, 6, 7, 8]
[0, 10, 11, 12]
[9, 13, 14, 15]

[5, 1, 3, 4]
[2, 6, 7, 8]
[9, 10, 11, 12]
[0, 13, 14, 15]

[5, 1, 3, 4]
[2, 6, 7, 8]
[9, 10, 11, 12]
[13, 0, 14, 15]

[5, 1, 3, 4]
[2, 6, 7, 8]
[9, 10, 11, 12]
[13, 14, 0, 15]

[5, 1, 3, 4]
[2, 6, 7, 8]
[9, 10, 0, 12]
[13, 14, 11, 15]

[5, 1, 3, 4]
[2, 6, 7, 8]
[9, 0, 10, 12]
[13, 14, 11, 15]

[5, 1, 3, 4]
[2, 0, 7, 8]
[9, 6, 10, 12]
[13, 14, 11, 15]

[5, 1, 3, 4]
[0, 2, 7, 8]
[9, 6, 10, 12]
[13, 14, 11, 15]

[0, 1, 3, 4]
[5, 2, 7, 8]
[9, 6, 10, 12]
[13, 14, 11, 15]

[1, 0, 3, 4]
[5, 2, 7, 8]
[9, 6, 10, 12]
[13, 14, 11, 15]

[1, 2, 3, 4]
[5, 0, 7, 8]
[9, 6, 10, 12]
[13, 14, 11, 15]

[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 0, 10, 12]
[13, 14, 11, 15]

[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 10, 0, 12]
[13, 14, 11, 15]

[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 10, 11, 12]
[13, 14, 0, 15]

[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 10, 11, 12]
[13, 14, 15, 0

In [ ]:
# your code here
